https://youtu.be/oqS-bR5m1bI

## Web scraping ( https://bus.gov.ru/registry ). Selenium library

https://chromedriver.chromium.org/downloads - ссылка для установки расширения для хрома. Необходимо для работы с библиотекой

In [62]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep

In [63]:
browser = Chrome("./chromedriver")

<ipython-input-63-e3422d95d342>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome("./chromedriver")


In [64]:
url = "https://bus.gov.ru/registry"

browser.get(url)

### Looking for the input field

In [65]:
input_tab = browser.find_element(By.XPATH, '/html/body/div[2]/ui-view/form/div[1]/div[1]/label/input')
input_tab

<selenium.webdriver.remote.webelement.WebElement (session="9670904715032afa6bdc820addd8abb3", element="a8f11d32-e042-45a5-9041-bda30903e609")>

### Writing "swimming pool" into the input field

In [66]:
input_tab.send_keys("бассейн")

### Clicking the "search" button

In [67]:
button = browser.find_element(By.XPATH, '//button[@type="submit"]')
button.click()

### The same action in another way: "clicking" Enter key

```python
input_tab.send_keys(Keys.ENTER)
```

## Collecting data from downloaded page's code using <i> bs4 </i> library

In [68]:
soup = BeautifulSoup(browser.page_source, 'lxml')

#### getting the name of an object from the code

In [69]:
name = soup.find('a', class_ = 'result__title').text.strip()
name

'АВТОНОМНОЕ УЧРЕЖДЕНИЕ "ПЛАВАТЕЛЬНЫЙ БАССЕЙН "АГАН" ГОРОДСКОГО ОКРУГА РАДУЖНЫЙ ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГРЫ'

#### getting the link of an object

In [70]:
link = "https://bus.gov.ru/registry" + soup.find('a', class_ = 'result__button_registry').get('href')
link

'https://bus.gov.ru/registry/agency/217441/register-info'

In [71]:
orgs = soup.findAll('div', class_ = 'result')
#orgs

In [72]:
len(orgs)

10

### there are 10 elements on each page. We are going to collect their data and switch to the new page etc

#### the first page:

In [73]:
data1 = []
for org in orgs:
    name = org.find('a', class_ = 'result__title').text.strip()
    link = "https://bus.gov.ru/registry" + soup.find('a', class_ = 'result__button_registry').get('href')
    data1.append([name, link])

In [74]:
data1

[['АВТОНОМНОЕ УЧРЕЖДЕНИЕ "ПЛАВАТЕЛЬНЫЙ БАССЕЙН "АГАН" ГОРОДСКОГО ОКРУГА РАДУЖНЫЙ ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГРЫ',
  'https://bus.gov.ru/registry/agency/217441/register-info'],
 ['БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ФИЗИЧЕСКОЙ КУЛЬТУРЫ И СПОРТА ШЕКСНИНСКОГО МУНИЦИПАЛЬНОГО РАЙОНА "ПЛАВАТЕЛЬНЫЙ БАССЕЙН "ДЕЛЬФИН"',
  'https://bus.gov.ru/registry/agency/217441/register-info'],
 ['ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ "БАССЕЙН "ОРИОН"',
  'https://bus.gov.ru/registry/agency/217441/register-info'],
 ['МУНИЦИПАЛЬНАЯ БЮДЖЕТНАЯ ОРГАНИЗАЦИЯ ДОПОЛНИТЕЛЬНОГО ОБРАЗОВАНИЯ ДЕТСКИЙ ОЗДОРОВИТЕЛЬНО-ОБРАЗОВАТЕЛЬНЫЙ ЦЕНТР (ПЛАВАТЕЛЬНЫЙ БАССЕЙН "ЯНТАРЬ") Г. ЗАДОНСКА ЛИПЕЦКОЙ ОБЛАСТИ',
  'https://bus.gov.ru/registry/agency/217441/register-info'],
 ['МУНИЦИПАЛЬНОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ БЕССОНОВСКОГО РАЙОНА ПЕНЗЕНСКОЙ ОБЛАСТИ "БАССЕЙН БЕССОНОВСКИЙ"',
  'https://bus.gov.ru/registry/agency/217441/register-info'],
 ['МУНИЦИПАЛЬНОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ КУПИНСКОГО РАЙОНА "ПЛАВАТЕЛЬНЫЙ БАССЕЙН "ЮБИЛЕЙНЫЙ"',
  'https://b

#### clicking on "next page" button for 3 times (dealing with 4 pages) and collecting the data:

# <font color="red"> Final Code </font>

In [78]:
data = [] # creating list for data

browser = Chrome("./chromedriver") # opening Chrome with a special driver
url = "https://bus.gov.ru/registry" # link for web scraping
browser.get(url) # opening this link with Chrome using a driver

input_tab = browser.find_element(By.XPATH, '/html/body/div[2]/ui-view/form/div[1]/div[1]/label/input') 
# looking for input field
input_tab.send_keys("бассейн") # writing smth into the input field
button = browser.find_element(By.XPATH, '//button[@type="submit"]') # looking for "search" button
button.click() # clicking on the "search" button


for p in range(4): # series for the page switching
    
    sleep(7) # waiting for the page updating for 7 seconds
    
    soup = BeautifulSoup(browser.page_source, 'lxml') # saving a new page's code
    
    orgs = soup.findAll('div', class_ = 'result') # saving a code of elements with needed data 
    
    for org in orgs:
        # looking for an organisation's name and link in the saved page code
        try:
            name = org.find('a', class_ = 'result__title').text.strip()
        except:
            name = "-"
        try:
            link = "https://bus.gov.ru/registry" + soup.find('a', class_ = 'result__button_registry').get('href')
        except:
            link = "-"
        # adding new data to list
        data.append([name, link])
    
    print("Загружено данных:", len(data))
    
    try: 
        browser.find_element(By.CLASS_NAME, 'pagination__next').click() # clicking "next page" button
    except:
        continue

<ipython-input-78-08783d5d6789>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome("./chromedriver") # открытие хрома с расширением


Загружено данных: 10
Загружено данных: 20
Загружено данных: 30
Загружено данных: 37


## Saving collected data as .csv file

In [80]:
import pandas as pd

In [82]:
header = ["Name", "Link"]

In [83]:
df = pd.DataFrame(data, columns = header)
df.to_csv("./busgov.csv", sep = ";", encoding = "utf8")